<a href="https://colab.research.google.com/github/seanjincmu/myprojects/blob/master/estimated_travel_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyproj
from pyproj import Proj

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
core_folder = "drive/My Drive/10701 Project/"
maps = pd.read_csv(core_folder+ "Maps/taxi_zone_lookup.csv")
#this reads in a pandas dataframe of the taxi_zone_lookup
taxi_folder = core_folder + "TaxiData"


In [0]:
import numpy as np
taxi_zones = core_folder + "Maps/taxi_zones_utm.csv"
utm = pd.read_csv(taxi_zones)
df_loc = utm[["LocationID", "X", "Y"]]
df_loc.head()
myProj = Proj(" +proj=lcc +lat_1=40.66666666666666 +lat_2=41.03333333333333 +lat_0=40.16666666666666 +lon_0=-74 +x_0=300000 +y_0=0 +datum=NAD83 +units=us-ft +no_defs")
x, y = np.array(df_loc["X"]), np.array(df_loc["Y"])
x, y = myProj(x, y, inverse = True)
x = x 
y = y
df_loc["lat"], df_loc["long"] = y, x
df_loc = df_loc.drop(columns = ["X", "Y"], axis = 1)
df_loc = df_loc.drop(df_loc.index[[263, 264]])
df_loc

,LocationID,lat,long
0,1,40.689516,-74.176786
1,2,40.625724,-73.826126
2,3,40.865888,-73.849479
3,4,40.724152,-73.977023
4,5,40.550340,-74.189930
...,...,...,...
258,259,40.899103,-73.856351
259,260,40.746798,-73.903713
260,261,40.708976,-74.012919
261,262,40.776534,-73.945830


In [0]:
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score
def errors(y_val, y_pred):
    mse = mean_squared_error(y_val, y_pred)
    msle = mean_squared_log_error(y_val, y_pred)

    print('Mean squared error: %.5f'
       % mse)
    
    print("Mean squared log error: %.5f" % msle)

    print('Root Mean squared error: %.5f'
       % np.sqrt(mse))
    
    print('Root Mean Squared Log Error: %.5f' % np.sqrt(msle))

    
    print('Coefficient of determination: %.5f'
       % r2_score(y_val, y_pred))
    

In [0]:
import random
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, neural_network
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [15, 10]

def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def random_longitude():
    return np.random.uniform(40.6, 40.9)

def random_latitude():
    return np.random.uniform(74.7, 74.1)

np.random.seed(1000)

def sample(x):
    global core_folder
    return (list(map(lambda y: core_folder+'TaxiData/assignment1_data-' + str(y) + '.csv', random.sample(range(1,674),x))))

def head():
    global core_folder
    return (list(map(lambda y: core_folder+'TaxiData/assignment1_data-' + str(y) + '.csv', np.arange(1,24))))


In [0]:
def trainer():
    train = pd.concat([pd.read_csv(f) for f in head()], ignore_index = True)
    print(train.columns)
    print('{} rows'.format(train.shape[0]))

    # rename datetime features so they look better and find duration
    train['pickup_datetime'] = pd.to_datetime(train.tpep_pickup_datetime)
    train['dropoff_datetime'] = pd.to_datetime(train.tpep_dropoff_datetime)
    del train['tpep_pickup_datetime']
    del train['tpep_dropoff_datetime']
    train['duration'] = (train['dropoff_datetime'] - train['pickup_datetime']).map(lambda x : x.total_seconds())
    train['PULocationID'] = train[train['PULocationID'] != 264]
    train['PULocationID'] = train[train['PULocationID'] != 265]
    train['DOLocationID'] = train[train["DOLocationID"] != 264]
    train['DOLocationID'] = train[train["DOLocationID"] != 265]

    # replace with actual data
    train['pickup_latitude'] = train['PULocationID'].map(lambda x : set_latitude(x))
    train['pickup_longitude'] = train['PULocationID'].map(lambda x : set_longitude(x))

    train['dropoff_latitude'] = train['DOLocationID'].map(lambda x : set_latitude(x))
    train['dropoff_longitude'] = train['DOLocationID'].map(lambda x : set_longitude(x))

    del train['PULocationID']
    del train['DOLocationID']

    train['distance_haversine'] = haversine_array(train['pickup_latitude'], train['pickup_longitude'], train['dropoff_latitude'], train['dropoff_longitude'])

    # Speed
    #train['avg_speed'] =  train['distance_haversine'] / train['duration'].map(lambda x : 0 if x > 10 ** 5 else 3600 * x)
    train = train[~train.isin([np.nan, np.inf, -np.inf]).any(1)]
    return train
    

In [0]:
def constant_estimator(data):
  train, test = train_test_split(data, test_size = 0.2)
  m = train["duration"].mean()

  error = test["duration"] - m

  error = error**2
  rmse = np.sqrt(error.sum())

  log_error = np.log(test["duration"] + 1)

  log_error = -log_error
  log_error = log_error + np.log(m + 1)
  rmsle = np.sqrt(log_error.sum())

  return (rmse, rmsle)

In [0]:
train = trainer()
constant_estimator(train)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'PULocationID', 'DOLocationID', 'payment_type'],
      dtype='object')
2299977 rows


(2221183.2105386266, 433.79384662678075)

In [0]:
#simple linear regression
def simpleRegress(train):
    y = train['duration'].values
    # XGB decision tree
    features = 'distance_haversine'
    print(train[features].shape)
    X_train, X_val, y_train, y_val = train_test_split(train[features].values, y, test_size=.2, random_state=1000)
    regr = linear_model.LinearRegression()


    # Train the model using the training sets
    regr.fit(X_train.reshape(-1,1), y_train)
    #network.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred_reg = regr.predict(X_val.reshape(-1,1))
    #y_pred_net = network.predict(X_val)

    
    # The coefficient of determination: 1 is perfect prediction
    errors(y_val, y_pred_reg)
    
    # The coefficient of determination: 1 is perfect prediction
    #print('Coefficient of determination: %.5f'
    #   % r2_score(y_val, y_pred_net))
  

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'PULocationID', 'DOLocationID', 'payment_type'],
      dtype='object')
2299977 rows
(2299977,)
Mean squared error: 10725429.90543
Mean squared log error: 0.85658
Root Mean squared error: 3274.97021
Root Mean Squared Log Error: 0.92551
Coefficient of determination: -0.00000


In [0]:
simpleRegress(train)

In [0]:
#multivariate linear regression
def multiRegress(train):
    y = train['duration'].values
    # XGB decision tree
    features = [f for f in train.columns if f not in ['pickup_datetime', 'dropoff_datetime', 'duration']]
    X_train, X_val, y_train, y_val = train_test_split(train[features].values, y, test_size=.2, random_state=1000)
    regr = linear_model.LinearRegression()
    network = neural_network.MLPRegressor()


    # Train the model using the training sets
    regr.fit(X_train, y_train)
    #network.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred_reg = regr.predict(X_val)
    #y_pred_net = network.predict(X_val)

    
    errors(y_val, y_pred_reg)



In [0]:
multiRegress(train)

Mean squared error: 10711113.04904
Mean squared log error: 0.85890
Root Mean squared error: 3272.78369
Root Mean Squared Log Error: 0.92677
Coefficient of determination: 0.00133


In [0]:
#simple linear regression
def neuralnet(train):
    y = train['duration'].values
    # XGB decision tree
    features = [f for f in train.columns if f not in ['pickup_datetime', 'dropoff_datetime', 'duration']]
    print(train[features].shape)
    X_train, X_val, y_train, y_val = train_test_split(train[features].values, y, test_size=.2, random_state=1000)
    network = neural_network.MLPRegressor(hidden_layer_sizes=(10, 9, 8, 7, 6 ))


    network.fit(X_train, y_train)

    y_pred_net = network.predict(X_val)

    
    # The coefficient of determination: 1 is perfect prediction
    errors(y_val, y_pred_reg)
    


In [0]:
neuralnet(train)

(2299977, 8)


NameError: ignored

In [0]:
# This is the main model for XGBoost. To run it, call tree() with corresponding parameters
import numpy as np
import random
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score, mean_squared_error
import xgboost as xgb
import os
from pyproj import Proj
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [15, 10]

np.random.seed(1000)

# Draws the XGBoost Tree
def plot_tree(xgb_model, filename, rankdir='UT'):
    gvz = xgb.to_graphviz(xgb_model, num_trees=2, rankdir=rankdir)
    _, file_extension = os.path.splitext(filename)
    format = file_extension.strip('.').lower()
    data = gvz.pipe(format=format)
    full_filename = filename
    with open(full_filename, 'wb') as f:
        f.write(data)

# Elbow method
def elbow(coords, sample_ind):
    Sum_of_squared_distances = []
    K = range(1,25)
    for k in K:
        kmeans = MiniBatchKMeans(n_clusters=k, batch_size=10000).fit(coords[sample_ind])
        Sum_of_squared_distances.append(kmeans.inertia_)

    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('number of clusters')
    plt.ylabel('SSE Distances')
    plt.title('Computing optimal number of clusters')
    plt.savefig('Choose_cluster.png')
    fig.clear()

print(df_loc)

def set_latitude(x):
    if x == 264 or x == 265:
        raise SystemExit(0)
    return df_loc['lat'][x-1]

def set_longitude(x):
    if x == 264 or x == 265:
        raise SystemExit(0)
    return -df_loc['long'][x-1]

# Use this to plot graphs from output txt
def txt_to_array(src):
    f = open('file_name.ext', 'r')
    x = f.read().split()
    f.close()
    vals = []
    for xx in x:
        vals.append(float(xx))
    return vals

# longitude and latitude range of nyc (for plotting)
nyc_long = (40.6, 40.9)
nyc_lat = (73.7, 74.1)

# main model, the parameters are XGBoost parameters
# sample is the number of samples
def tree(name, max_depth, rounds, min_child_weight, eta, subsample, samples):
    # remove useless stuff
    train = pd.concat([pd.read_csv(f) for f in sample(samples)])
    train['PULocationID'].replace(264, np.nan, inplace=True)
    train['PULocationID'].replace(265, np.nan, inplace=True)
    train['DOLocationID'].replace(264, np.nan, inplace=True)
    train['DOLocationID'].replace(265, np.nan, inplace=True)
    train.replace('', np.nan, inplace=True)
    train.dropna(subset=['PULocationID'], inplace=True)
    train.dropna(subset=['DOLocationID'], inplace=True)
    train.dropna(inplace=True)

    print('{} rows'.format(train.shape[0]))

    # rename datetime features so they look better and find duration
    train['pickup_datetime'] = pd.to_datetime(train.tpep_pickup_datetime)
    train['dropoff_datetime'] = pd.to_datetime(train.tpep_dropoff_datetime)
    del train['tpep_pickup_datetime']
    del train['tpep_dropoff_datetime']
    train['duration'] = (train['dropoff_datetime'] - train['pickup_datetime']).map(lambda x : x.total_seconds())
    train['log_duration'] = train['duration'].map(lambda x : np.log(x))


    # replace with actual data
    train['pickup_latitude'] = train['PULocationID'].map(lambda x : set_latitude(x))
    train['pickup_longitude'] = train['PULocationID'].map(lambda x : set_longitude(x))

    train['dropoff_latitude'] = train['DOLocationID'].map(lambda x : set_latitude(x))
    train['dropoff_longitude'] = train['DOLocationID'].map(lambda x : set_longitude(x))

    del train['PULocationID']
    del train['DOLocationID']

    '''
    fig, ax = plt.subplots(ncols=2, figsize=(10,5))
    ax[0].scatter(train['pickup_latitude'].values, train['pickup_longitude'].values,
                color='red', s=1, alpha=0.1)
    ax[1].scatter(train['dropoff_latitude'].values, train['dropoff_longitude'].values,
                color='blue', s=1, alpha=0.1)
    fig.suptitle('Coordinates')
    ax[0].set_ylabel('longitude')
    ax[0].set_xlabel('latitude')
    ax[1].set_ylabel('longitude')
    ax[1].set_xlabel('latitude')
    ax[0].set_xlim(nyc_lat)
    ax[0].set_ylim(nyc_long)
    ax[1].set_xlim(nyc_lat)
    ax[1].set_ylim(nyc_long)
    plt.savefig('Coord.png')
    fig.clear()'''

    train['distance_haversine'] = haversine_array(train['pickup_latitude'], train['pickup_longitude'], train['dropoff_latitude'], train['dropoff_longitude'])
    train['log_distance'] = train['distance_haversine'].map(lambda x : np.log(x + 1))

    # PCA
    xys = np.vstack((train[['pickup_latitude', 'pickup_longitude']].values,
                     train[['dropoff_latitude', 'dropoff_longitude']].values))
    pca = PCA().fit(xys)
    train['pickup_pca0'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 0]
    train['pickup_pca1'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 1]
    train['dropoff_pca0'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
    train['dropoff_pca1'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

    '''
    fig, ax = plt.subplots(ncols=2, figsize=(10,5))
    ax[0].scatter(train['pickup_pca0'].values, train['pickup_pca1'].values,
                color='red', s=1, alpha=0.1)
    ax[1].scatter(train['dropoff_pca0'].values, train['dropoff_pca1'].values,
                color='blue', s=1, alpha=0.1)
    fig.suptitle('PCA transformed coordinates of Pickup vs Dropoff')
    ax[0].set_ylabel('pca1')
    ax[0].set_xlabel('pca0')
    ax[1].set_ylabel('pca1')
    ax[1].set_xlabel('pca0')
    pca_borders = pca.transform([[x, y] for x in nyc_lat for y in nyc_long])
    ax[0].set_xlim(pca_borders[:, 0].min(), pca_borders[:, 0].max())
    ax[0].set_ylim(pca_borders[:, 1].min(), pca_borders[:, 1].max())
    ax[1].set_xlim(pca_borders[:, 0].min(), pca_borders[:, 0].max())
    ax[1].set_ylim(pca_borders[:, 1].min(), pca_borders[:, 1].max())
    plt.savefig('PCA.png')
    fig.clear()'''

    # Datetime
    train['pickup_date'] = train['pickup_datetime'].dt.date
    train['pickup_weekday'] = train['pickup_datetime'].dt.weekday
    train['pickup_hour_week_year'] = train['pickup_datetime'].dt.weekofyear
    train['pickup_hour'] = train['pickup_datetime'].dt.hour
    train['pickup_minute'] = train['pickup_datetime'].dt.minute

    # Speed
    train['avg_speed'] = train['distance_haversine'] / train['duration']

    '''
    fig, ax = plt.subplots(ncols=3, figsize=(15,5))
    ax[0].plot(train['pickup_hour'], train['avg_speed'], 'bo', lw=2, alpha=0.7)
    ax[1].plot(train['pickup_weekday'], train['avg_speed'], 'go', lw=2, alpha=0.7)
    ax[2].plot(train['pickup_week_hour'], train['avg_speed'], 'ro', lw=2, alpha=0.7)
    ax[0].set_xlabel('hour')
    ax[1].set_xlabel('weekday')
    ax[2].set_xlabel('weekhour')
    ax[0].set_ylabel('speed')
    fig.suptitle('Speed')
    plt.savefig('Speed.png')
    fig.clear()'''

    # Clustering
    kmeans = MiniBatchKMeans(n_clusters=8, batch_size=10000).fit(xys)
    train['pickup_cluster'] = kmeans.predict(train[['pickup_latitude', 'pickup_longitude']])
    train['dropoff_cluster'] = kmeans.predict(train[['dropoff_latitude', 'dropoff_longitude']])

    # elbow(coords, sample_ind)
    '''
    fig, ax = plt.subplots(ncols=2, figsize=(10,5))
    ax[0].scatter(train.pickup_longitude.values, train.pickup_latitude.values, s=10, lw=0,
            c=train.pickup_cluster.values, cmap='viridis')
    ax[0].set_xlim(nyc_long)
    ax[0].set_ylim(nyc_lat)
    ax[0].set_xlabel('longitude')
    ax[0].set_ylabel('latitude')
    ax[1].scatter(train.dropoff_longitude.values, train.dropoff_latitude.values, s=10, lw=0,
            c=train.dropoff_cluster.values)
    ax[1].set_xlim(nyc_long)
    ax[1].set_ylim(nyc_lat)
    ax[1].set_xlabel('longitude')
    ax[1].set_ylabel('latitude')
    fig.suptitle('Kmeans Pickup vs Dropoff')
    plt.savefig('Clustering.png')
    fig.clear()'''

    # datetime features are just strings, while duration is what we want to predict
    features = [f for f in train.columns if f not in ['pickup_datetime', 'dropoff_datetime', 'pickup_date', 'duration', 'log_duration']]
    print('We have %i features.' % len(features))
    y = train['duration'].values

    # XGB decision tree
    X_train, X_val, y_train, y_val = train_test_split(train[features].values, y, test_size=.2, random_state=1000)
    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
    dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)
    watchlist = [(dtrain, 'train'), (dval, 'val')]

    progress = dict()
    xgb_pars = {'max_depth': max_depth, 'eta': eta, 'subsample': subsample, 'min_child_weight': min_child_weight,
                'colsample_bytree': 0.5, 'lambda': 1., 'booster' : 'gbtree', 'silent': 1, 'eval_metric': 'rmse', 'objective': 'reg:linear'}

    model = xgb.train(xgb_pars, dtrain, int(rounds), watchlist, early_stopping_rounds=50, maximize=False, verbose_eval=10, evals_result=progress)
    
    '''
    fig, ax = plt.subplots(ncols=2, figsize=(10,5))
    ax[0].plot(np.arange(len(progress['train']['rmse'])), np.array(progress['train']['rmse']))
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('RMSE')
    ax[1].plot(np.arange(len(progress['val']['rmse'])), np.array(progress['val']['rmse']))
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('RMSE')
    fig.suptitle('Train vs Validation RMSE')
    plt.savefig('error.png')'''
    
    # save rmses to txt
    np.savetxt(name + str(max_depth) + str(rounds) + str(eta) + str(subsample) + "_train.txt", progress['train']['rmse'])
    np.savetxt(name + str(max_depth) + str(rounds) + str(eta) + str(subsample) + "_val.txt", progress['val']['rmse'])

    print('Final rmse %f' % model.best_score)

    return model.best_score

ModuleNotFoundError: ignored

In [0]:
max_depth = [7,8,9]
min_child_weight = [30,40,50]
eta = [0.1, 0.3, 0.5]
subsample = [0.5,0.75]

best_score = 100000
best_combination = []
for a in max_depth:
  for b in min_child_weight:
    for c in eta:
      for d in subsample:
        score = tree("search",a,100,b,c,d,100)
        if score < best_score:
          best_score = score
          best_combination = [a,b,c,d]

print(best_combination)
print(best_score)

2299977 rows


ValueError: ignored

In [0]:
L = []

In [0]:
for i in range(0, 8):
  L.append([])
  for j in range(0, 8):
    if (i % 2 == 0) and  (j == (i//2)):
      L[i].append(1)
    elif (i % 2 == 1) and (j == (i//2) + 4):
      L[i].append(1)
    else:
      L[i].append(0)
  

In [0]:
T =  []
for i in range(0, 8):
  T.append([])
  for j in range(0, 8):
    if i >= 4:
      if j >= 4:
        T[i].append(0)
      else:
        if (i == j + 4):
          T[i].append(3)
        else:
          T[i].append(0)
    else:
      if j >= 4:
        if (i == j - 4):
          T[i].append(2)
        else:
          T[i].append(0)
      else:
        if (i == j):
          T[i].append(1)
        else:
          T[i].append(0)
      
        
        


In [0]:
T

[[1, 0, 0, 0, 2, 0, 0, 0],
 [0, 1, 0, 0, 0, 2, 0, 0],
 [0, 0, 1, 0, 0, 0, 2, 0],
 [0, 0, 0, 1, 0, 0, 0, 2],
 [3, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 3, 0, 0, 0, 0]]

In [0]:
L_np = np.array(L)
T_np = np.array(T)

In [0]:
np.dot(L_np, np.dot(T_np, (np.transpose(L_np))))

array([[1, 2, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 2, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 0, 0],
       [0, 0, 0, 0, 3, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 2],
       [0, 0, 0, 0, 0, 0, 3, 0]])